In [35]:
pwd

'/Users/bhikipallai/Desktop/Projects'

In [36]:
ls

ETE_Mlops/                     dvcdemo/
Kserve/                        kidney_Disease_Classification/
Local_folder_dvc/              simple_app/


In [37]:
cd kidney_Disease_Classification

/Users/bhikipallai/Desktop/Projects/kidney_Disease_Classification


In [38]:
pwd

'/Users/bhikipallai/Desktop/Projects/kidney_Disease_Classification'

In [22]:
import dagshub
dagshub.init(repo_owner='Vicky7873', repo_name='kidney_Disease_Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "Vicky7873/kidney_Disease_Classification"

[2024-06-28 09:22:04,276: INFO: helpers: Initialized MLflow to track repo "Vicky7873/kidney_Disease_Classification"]


Repository Vicky7873/kidney_Disease_Classification initialized!

[2024-06-28 09:22:04,277: INFO: helpers: Repository Vicky7873/kidney_Disease_Classification initialized!]


In [46]:
import tensorflow as tf

In [47]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [48]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [49]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Vicky7873/kidney_Disease_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [50]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [51]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [52]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-06-28 09:31:23,897: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-28 09:31:23,901: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 09:31:23,902: INFO: common: created directory at: artifacts]
[2024-06-28 09:31:24,096: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.7800 - loss: 1.6685
[2024-06-28 09:31:38,073: INFO: common: json file saved at: scores.json]


2024/06/28 09:31:39 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/06/28 09:31:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
